<a href="https://colab.research.google.com/github/PraveenKumar-pk-star/GenAI_RAG/blob/main/multi_document_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install langchain
! pip install openai
! pip install langchain-openai
! pip install langchain-community
! pip install chromadb
! pip install  pypdf
! pip install langchain-chorma



  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.3
    Uninstalling langchain-core-1.0.3:
      Successfully uninstalled langchain-core-1.0.3
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 1.0.0
    Uninstalling langchain-text-splitters-1.0.0:
      Successfully uninstalled langchain-text-splitters-1.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.
langchain-community 0.

In [10]:
!pip uninstall -y langchain-core
! pip uninstall -y langchain-openai


Found existing installation: langchain-core 0.3.79
Uninstalling langchain-core-0.3.79:
  Successfully uninstalled langchain-core-0.3.79
Found existing installation: langchain-openai 0.3.35
Uninstalling langchain-openai-0.3.35:
  Successfully uninstalled langchain-openai-0.3.35


In [12]:
! pip install langchain-core==0.3.79
! pip install langchain-openai==0.3.35

In [4]:
from getpass import getpass
Open_Api_Key=getpass()


··········


In [5]:
from google.colab import files
uploaded=files.upload()

Saving custumer_service_support.pdf to custumer_service_support.pdf
Saving Img_Pdf_Rag.pdf to Img_Pdf_Rag (1).pdf


In [6]:
!pip install unstructured pytesseract pdf2image pillow
!apt-get install poppler-utils tesseract-ocr
!pip install pypdf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.11).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [8]:
! pip show langchain_openai

Name: langchain-openai
Version: 0.3.35
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


In [17]:
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import os
from openai import OpenAI
# from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [24]:
pdf_img_path=""

def extract_text_from_image(pdf_img_path):
    images = convert_from_path(pdf_img_path, dpi=150, first_page=1, last_page=5)
    full_text = ""
    for i, image in enumerate(images):
        print(f"Processing page {i+1}/{len(images)}...")
        text = pytesseract.image_to_string(image)
        full_text += text + "\n"
    return full_text

In [25]:
text_path=""
def extract_text_file (text_path):
  doc=PyPDFLoader(text_path)
  page=doc.load()
  return page

In [27]:
image_path="/content/Img_Pdf_Rag.pdf"
text_path="/content/earth_full_pdf.pdf"
image=extract_text_from_image(image_path)
text=extract_text_file(text_path)

Processing page 1/3...
Processing page 2/3...
Processing page 3/3...


In [28]:
from langchain_core.documents import Document
combine_docs=text+[Document(page_content=image)]

In [30]:
spliter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
embeeding=OpenAIEmbeddings(api_key=Open_Api_Key)
vectordb=Chroma.from_documents(documents=combine_docs,embedding=embeeding,persist_directory="IMG_pdf_store")
ret=vectordb.as_retriever()

In [34]:
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [40]:
prompt_Qa= """Act as AI assisant answer the question with only the following context below
if there is no answer in the context, "you are asking the irrevalent question The information is not provided in the document."
Context:
{context}

Question:
{input}
"""
prompt_template =PromptTemplate(template=prompt_Qa,input_variables=["context","input"])
llm = ChatOpenAI(api_key=Open_Api_Key, model="gpt-4o-mini")

# ---------------------------
# 5️⃣ Create Base Chain (no memory yet)
# ---------------------------
question_answer_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt_template
)

# ---------------------------
# 6️⃣ Create the retrieval chain (RAG logic)
# ---------------------------
retrieval_chain = create_retrieval_chain(
    retriever=ret,
    combine_docs_chain=create_stuff_documents_chain(
        llm=llm,
        prompt=prompt_template
    )
).pick("answer")


In [42]:
store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# --- Chain with Memory ---
chain_with_history = RunnableWithMessageHistory(
    retrieval_chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

print(" Chatbot ready! Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")

    if user_input.lower() == "exit":
        print("👋 Goodbye!")
        break

    response = chain_with_history.invoke(
    {"input": user_input},
    config={"configurable": {"session_id": "user1"}}
)
    print("\nAssistant:", response)

🤖 Chatbot ready! Type 'exit' to stop.

You: what is document about

Assistant: The document is about the geology of Earth as a planet, discussing its properties, origin, interior, atmosphere, and its comparison with other terrestrial planets in the solar system. It also includes information on Earth's unique ability to support life and the methodologies used to study its geology, including the investigation of its interior through seismic waves.
You: who is chef minister of tamilnadu

Assistant: you are asking the irrevalent question The information is not provided in the document.
You: what is the population of pople in  the   hillsboroguht county

Assistant: The total population of Hillsborough County, Florida is 1,459,762.
You: give the summary of the account charges

Assistant: You are asking the irrelevant question. The information is not provided in the document.
You: what is the summary of the account charges 

Assistant: You are asking the irrevalent question The information is

In [45]:
# store={}
# def get_session_history(session_id_1:str):
#   if session_id_1 not in store:
#     store[session_id_1]=InMemoryChatMessageHistory()
#     return store[session_id_1]


# chain_with_history=RunnableWithMessageHistory(
#     retrieval_chain,
#     get_session_history=get_session_history,
#     input_messages_key="input",
#     history_messages_key="chat_history"
# )

# while True:
#   user_input=input("You:")
#   if user_input.lower()=="exit":
#     break;
#   chat_bot=chain_with_history.invoke(
#       {"input":user_input},
#       confg={"configurable":{"session_id_1":"user_2"}}
#   )
#   print("answer:",chat_bot)


You:what is docuemt about


ValueError: Missing keys ['session_id'] in config['configurable'] Expected keys are ['session_id'].When using via .invoke() or .stream(), pass in a config; e.g., chain.invoke({'input': 'foo'}, {'configurable': {'session_id': '[your-value-here]'}})